In [123]:
import pygad 
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

In [121]:
expected_returns = pd.Series({
'ACWI': 0.07/12,
'^TBSP': 0.049/12,
'XAUPLN': 0.05/12,
'MWIG40': 0.065/12,
'IHYU.L': 0.038/12,
'PLOPLN3M': 0.03/12,
'edo': 0.04/12
})

cov = np.array([[0.001265,	0.000042],
            	[0.000042,	0.000164]])

In [167]:
assets = ['ACWI', '^TBSP']
means = expected_returns[assets]
number_of_scenarios = 1001
T = 60
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)
sim_returns.shape

(1001, 60, 2)

In [125]:
class Simulation():
    def __init__(self, initial_value, inflows, returns, strategy) -> None:
        self.capital = initial_value
        self.returns = returns
        self.strategy = strategy 
        self.inflows = inflows
    
   
    def step(self, current_value,inflow,weights,returns):
        current_assets_value = np.expand_dims(current_value,axis=0).T * weights
        next_assets_value =  np.round(current_assets_value * (1+returns),2)
        return next_assets_value.sum(1)
    
        
    def run(self, T):
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],self.strategy[t],self.returns[:,t])


In [147]:
portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,0.0]]

In [149]:
solution = [9,9,9,7,7,5,5,3,3,2,2]
np.take(portfolios, solution,axis=0)

array([[0.9, 0.1],
       [0.9, 0.1],
       [0.9, 0.1],
       [0.7, 0.3],
       [0.7, 0.3],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.2, 0.8],
       [0.2, 0.8]])

In [174]:
r = 0.9
T = 60
def fitness(ga_instance, solution, solution_idx):
    weights = np.take(portfolios, solution,axis=0)
    portfolio_sim = Simulation(100,np.full(60,0),sim_returns, weights)
    portfolio_sim.run(T)
    return np.quantile(portfolio_sim.capital,r)
    

In [175]:
solution = np.full(60,8)
fitness(None,solution,0)

178.77